In [10]:
import pandas as pd
import numpy as np
from pathlib import Path


In [12]:
from fastai import *

  Using cached fastai-1.0.58-py3-none-any.whl (236 kB)
  Using cached Bottleneck-1.3.7-cp310-cp310-macosx_10_9_universal2.whl
  Using cached pynvx-1.0.0.tar.gz (150 kB)
  Preparing metadata (setup.py) ... done
  Using cached numexpr-2.8.4-cp310-cp310-macosx_11_0_arm64.whl (89 kB)
  Using cached nvidia-ml-py3-7.352.0.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for pynvx, since package 'wheel' is not installed.
Using legacy 'setup.py install' for nvidia-ml-py3, since package 'wheel' is not installed.
  Running setup.py install for pynvx ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for pynvx did not run successfully.
  │ exit code: 1
  ╰─> [66 lines of output]
      running install
      /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.


In [13]:
path = Path('/')
np.random.seed(42)
data = ImageDataBunch.from_df(path = path, df = labels_train, folder = 'train', valid_pct=0.2, suffix='.jpg', num_workers=0)

NameError: name 'ImageDataBunch' is not defined

In [ ]:
data.classes

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-4,1e-3))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,2e-6))

In [ ]:
learn.save('stage-3')

Making Predictions

In [ ]:
with open('output.csv',"w") as file:
    file.write("ID,Prediction\n")
    for file in labels_test.filename:
        img = open_image('test/'+str(file)+'.jpg')
        prediction = str(learn.predict(img)[0]).split()[0]
        file.write(str(file)+','+prediction)
        file.write('\n')

In [ ]:
output = pd.read_csv('output.csv')
output.head()

Cleanup

In [ ]:
'''
%%capture
!apt-get install zip
!zip -r train.zip /kaggle/working/train/
!zip -r test.zip /kaggle/working/test/
!rm -rf train/*
!rm -rf test/*
'''